Semantic search index

In [13]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch

In [6]:
dataset = load_dataset('multi_news', split="test")
df = dataset.to_pandas().sample(2000, random_state=42)
df

Found cached dataset multi_news (C:/Users/cristian.sanchezp/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


,document,summary
4830,Tweet with a location \n \n You can add locati...,– Denis Finley has taken to Twitter to call Po...
1255,CNN host Piers Morgan just called to discuss h...,– CNN's Piers Morgan thinks gun-rights propone...
80,White House communications director Anthony Sc...,– New White House communications director Anth...
3044,CLOSE Scientists say they've found archaeologi...,– Scientists say they have the first physical ...
4486,Click image above to view graphic \n \n Althou...,– Scientists are calling it a breakthrough and...
...,...,...
2157,"On Thursday afternoon, President-elect Donald ...","– He who pays the piper calls the tune, and it..."
3615,Donald Trump said Sunday that in the wake of t...,– In the wake of the Orlando shooting one week...
2751,Nashua police believe body found is that of mi...,"– Sad news out of Nashua, NH, after police say..."
622,The public school systems in New York and Los ...,"– Some 640,000 kids in the nation's second-lar..."


In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
passage_embeddings = list(model.encode(df['summary'].to_list(), show_progress_bar=True))

Batches: 100%|██████████| 63/63 [45:17<00:00, 43.13s/it]


In [9]:

# Print the shape of the first passage embedding
passage_embeddings[0].shape

(384,)

In [10]:

# Declare a query string
query = "Find me some articles about technology and artificial intelligence"

In [11]:

# Define a function to find relevant news articles based on a given query
def find_relevant_news(query):
    # Encode the query using the sentence transformer model
    query_embedding = model.encode(query)
    # Print the shape of the query embedding
    query_embedding.shape

    # Calculate the cosine similarity between the query embedding and the passage embeddings
    similarities = util.cos_sim(query_embedding, passage_embeddings)

    # Find the indices of the top 3 most similar passages
    top_indicies = torch.topk(similarities.flatten(), 3).indices

    # Get the top 3 relevant passages by slicing the summaries at 200 characters and adding an ellipsis
    top_relevant_passages = [df.iloc[x.item()]['summary'][:200] + "..." for x in top_indicies]

    # Return the top 3 relevant passages
    return top_relevant_passages

In [17]:
find_relevant_news("Programming")

['– America can once again boast that it is home to the world\'s fastest supercomputer. IBM\'s Sequoia system knocked Japan\'s K Computer into second place after clocking in at a staggering 16.32 "petaflop...',
 '– The "tech surge" to fix HealthCare.gov includes some names from the industry\'s biggest players. Among them, per a Health department blog post, is Michael Dickerson, on leave from his job as a site r...',
 '– Are you a "digital native" or a "digital immigrant," and does it make a difference? Research recently published in the Teaching and Teacher Education journal indicates the concept of so-called digit...']